In [1]:
from pathlib import Path
import pickle
import itertools

import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from ruben import PersistenceDiagram

In [14]:
from gtda.diagrams import PersistenceEntropy, ComplexPolynomial

def features_for_dim(pd, dim):
	# b: birth, d: death, q: dimension
	b, d, _ = pd[pd[:,2] == dim].T
	if dim == 0:
		assert d[-1] == np.inf
		d[-1] = 1
	with np.errstate(invalid='ignore', divide='ignore'):
		return [
			mean_bd := np.c_[b, d].mean(axis=0),
			mean_bd**2,
			np.nan_to_num(1/mean_bd + np.log(mean_bd)), # fix divide by zero
			np.c_[b, d].std(axis=0),
			[np.mean(b - d)],
			[np.mean(b - d)**2],
			[np.mean((b + d) / 2)],
			[np.mean((b + d) / 2)**2],
			PersistenceEntropy().fit_transform([np.c_[pd]])[0],
			ComplexPolynomial().fit_transform([np.c_[pd]])[0],
			# should i include the other stuff
			np.pad(np.sort(d - b)[:-11:-1], (0, max(0, 10 - len(b))))
		]

feature_names = [
	'avg_birth_death', 'avg_birth_death_squared', 'avg_birth_death_inverted', 'std_birth_death',
	'avg_life', 'avg_life_squared', 'avg_half_life', 'avg_half_life_squared',
	'entropy',
	'complex_roots',
	'pooling'
]

def features_all_dims(pd):
	arrs = [np.concatenate(features_for_dim(pd, dim)) for dim in (0, 1)]
	return np.vstack(arrs)

In [15]:
outdir = Path('../out/task1/random')

def extract_pd(path: Path):
	with open(path, 'rb') as f:
		obj: PersistenceDiagram = pickle.load(f)
	return np.array([[p.birth, p.death, p.dim] for p in obj.points])

pds = [*map(extract_pd, outdir.glob('*.bin'))]

In [16]:
lengths = map(len, features_for_dim(pds[0], 0))
feature_lengths = dict(zip(feature_names, lengths))

In [17]:
X = np.stack([*map(features_all_dims, tqdm(pds))])

100%|██████████| 90/90 [04:45<00:00,  3.17s/it]


In [27]:
X = np.nan_to_num(X)

In [22]:
import json


ref_file = Path(
    "/Users/otis/Documents/rubens_speelhoekje/google/google_data/public_data/reference_data/task1_v4/model_configs.json")
config = json.loads(ref_file.read_text())
y = [
	config[name]['metrics']['train_acc'] - config[name]['metrics']['test_acc']
	for f in outdir.glob('*.bin') if (name := f.stem.split('_')[1])
]

In [28]:
def feature_mask(included):
	return np.concatenate([
		np.full(length, 1 if feature in included else 0)
		for feature, length in feature_lengths.items()
	])

def run_experiment(X, y, model_fn):
	results = {}
	combinations = itertools.chain(*(
		itertools.combinations(feature_names, r)
		for r in range(1, len(feature_names) + 1)
	))
	for features in tqdm([*combinations]):
		# Should I do cross-validation here?
		# and vary the splits over the different combos
		# or keep them the same?
		X_masked = X[:,:,np.where(feature_mask(features))[0]]
		X_flattened = X_masked.reshape((len(X), -1))
		X_train, X_test, y_train, y_test = \
			train_test_split(X_flattened, y, train_size=0.7)
		reg = model_fn().fit(X_train, y_train)
		results[features] = reg.score(X_test, y_test)
	return results

In [30]:
results = run_experiment(X, y, LinearRegression)

100%|██████████| 2047/2047 [00:06<00:00, 334.84it/s]


In [32]:
import operator


max(results.items(), key=operator.itemgetter(1))

(('avg_birth_death',
  'avg_birth_death_squared',
  'std_birth_death',
  'avg_life',
  'avg_life_squared',
  'avg_half_life',
  'avg_half_life_squared'),
 0.9233337809975324)

In [47]:
from tqdm import tqdm

def extract_pd(path: Path):
	with open(path, 'rb') as f:
		obj: PersistenceDiagram = pickle.load(f)
	pd = np.array([[p.birth, p.death, p.dim] for p in obj.points])

X = np.array(list(map(extract_features, tqdm(paths))))

100%|██████████| 96/96 [00:29<00:00,  3.31it/s]


In [48]:
import json

model_names = [p.stem.partition('_')[-1] for p in paths]
with open(DATA_PATH / '../../reference_data/task1_v4/model_configs.json') as f:
	config = json.load(f)

y = [config[name]['metrics']['train_acc'] - config[name]['metrics']['test_acc'] for name in model_names]

In [64]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge

X_train, X_test, y_train, y_test = train_test_split(
	X, y, train_size=0.7, random_state=20)

Ridge(alpha=0.5).fit(X_train, y_train).score(X_test, y_test)

/Users/otis/Sync/uni/tfm/.venv/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:251: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(


0.023784005473514114